In [1]:
import pickle
import os
import re
import numpy as np
import spacy
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datetime import datetime
from sklearn.preprocessing import normalize as scikit_normalize
from evaluation import plot_history
from evaluation import rmse_report
from sampling import UnderSampler3D
from fasttext_embedding import FastTextEmbeddingBag
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm

%matplotlib inline
%load_ext tensorboard

In [2]:
FROM_CHECKPOINT=True
MODEL_SAVE_DIR='/home/kvassay/data/z/models/E8/keras_cnn.h5'
MODEL_CHECKPOINT_PATH='/home/kvassay/data/z/models/E8/checkpoint/cp-{epoch:04d}.ckpt'

DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'
TB_LOG_DIR='/home/kvassay/data/z/log/E8/scalars/'
VEC_DIM=100
FASTTEXT='/home/kvassay/data/z/models/fasttext/cbow_{}_e{}_w{}.bin'.format(VEC_DIM,50,5)
SEQ_PADDING=50
CHECKPOINT_DIR='/tmp/z/checkpoint_dim{}_pad{}/'.format(VEC_DIM,SEQ_PADDING)
ALLOWED_SPECIAL=tuple(['?','!',':(', ':)', ':D',':-)',':-D',':\'(',':/',':-/','<3',':-P',':P'])

## Read data

In [3]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,dev,_=pickle.load(f)

CPU times: user 4.59 s, sys: 1.17 s, total: 5.76 s
Wall time: 5.74 s


## Load models

In [4]:
%%time
if not FROM_CHECKPOINT:
    fasttext=FastTextEmbeddingBag(FASTTEXT)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs


## Preprocess text + extract features
- filter out EN stop-words (and, or, ...)
- filter out non-allowed special tokens (we want to keep smileys and !,?)

In [ ]:
rx_special= re.compile("|".join(re.escape(s) for s in ALLOWED_SPECIAL))

def word_filter(word):
    if word in STOP_WORDS:
        return False
    if not word.isalpha():
        if not rx_special.findall(word):
            return False
    else:
        if len(word)<3:
            return False
    return True

def preprocess_text(text):
    return [x.lower() for x in text if word_filter(x.lower())]

def preprocess_texts(dataset,text_keys=['summary','text']):
    for sample in tqdm(dataset):
        for key in text_keys:
            sample[key]=preprocess_text(sample[key])
    return dataset

In [ ]:
%%time
if not FROM_CHECKPOINT:
    train=preprocess_texts(train)
    dev=preprocess_texts(dev)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.4 µs


## Extract features
- transform texts to averages of their fastText vectors
- concatenate summary & text average vectors into single one

In [ ]:
def pad_vecs(vecs_mtx,length):
    return pad_sequences(vecs_mtx,
                         maxlen=length,
                         dtype='float32',
                        padding='post',
                        truncating='post')

def extract_features(dataset, fasttext):
    default_vec=np.zeros(VEC_DIM,dtype=np.float32)
    vecs_all=[]
    for sample in tqdm(dataset):
        all_words=sample['summary']+sample['text']
        if all_words:
            vecs=fasttext.forward([x for x in all_words])
        else:
            vecs=np.array([default_vec])
        vecs=scikit_normalize(vecs)
        vecs=vecs.reshape(1,vecs.shape[0],vecs.shape[1])         
        vecs = pad_vecs(vecs, SEQ_PADDING)
        vecs_all.append(vecs)
    vecs_all=np.array(vecs_all)
    vecs_all=vecs_all.reshape(vecs_all.shape[0],vecs_all.shape[2],vecs_all.shape[3])
    return vecs_all

In [ ]:
def load_checkpoint():
    with open(CHECKPOINT_DIR+'X_train.npy','rb') as f:
        X_train=np.load(f)
    with open(CHECKPOINT_DIR+'X_dev.npy','rb') as f:
        X_dev=np.load(f)
    with open(CHECKPOINT_DIR+'y_train.npy','rb') as f:
        y_train=np.load(f)
    with open(CHECKPOINT_DIR+'y_dev.npy','rb') as f:
        y_dev=np.load(f)
    return X_train,X_dev,y_train,y_dev
        
def checkpoint(X_train,X_dev,y_train,y_dev):
    if not os.path.exists(CHECKPOINT_DIR):
        os.makedirs(CHECKPOINT_DIR)
    with open(CHECKPOINT_DIR+'X_train.npy','wb') as f:
        np.save(f,X_train)
    with open(CHECKPOINT_DIR+'X_dev.npy','wb') as f:
        np.save(f,X_dev)
    with open(CHECKPOINT_DIR+'y_train.npy','wb') as f:
        np.save(f,y_train)
    with open(CHECKPOINT_DIR+'y_dev.npy','wb') as f:
        np.save(f,y_dev)

In [ ]:
%%time
if not FROM_CHECKPOINT:
    X_train=extract_features(train,fasttext)
    X_dev=extract_features(dev,fasttext)
    y_train=np.array([x['score'] for x in train])
    y_dev=np.array([x['score'] for x in dev])
    print('Train samples shape: {}, Dev samples shape: {}'.format(X_train.shape,X_dev.shape))
else:
    X_train,X_dev,y_train,y_dev=load_checkpoint()

CPU times: user 87.3 ms, sys: 5.98 s, total: 6.07 s
Wall time: 22.8 s


In [ ]:
%%time
if not FROM_CHECKPOINT:
    checkpoint(X_train,X_dev,y_train,y_dev)

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 5.72 µs


## Experiment

In [ ]:
def experiment(learning_rate,epochs,batch_size,name,steps):
    model=train_model(epochs=epochs,batch_size=batch_size,learning_rate=learning_rate,steps=steps)
    y_pred_dev=model.predict(X_dev)
    rmse_report(y_dev,y_pred_dev,title='{} - RMSE report'.format(name))
    plot_history(model,title='{} - Train/Dev MSE'.format(name))
    return model

In [ ]:
def get_tb_callback():
    suffix=datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir= os.path.join(TB_LOG_DIR,suffix)
    return K.callbacks.TensorBoard(log_dir=os.path.join(log_dir))

def penalized_loss(y_true, y_pred):
    return K.backend.mean(K.backend.square(K.backend.abs(y_true - y_pred))/y_true)

def train_model(batch_size,learning_rate, epochs,steps):
    DROPRATE=0.05
    
    cp_callback = tf.keras.callbacks.ModelCheckpoint(MODEL_CHECKPOINT_PATH,
                                                     verbose=1,
                                                     save_weights_only=False,
                                                     period=50)
    
    tensorboard_callback = get_tb_callback()
    model = K.models.Sequential([
        K.layers.Conv1D(256, 3, activation='relu',strides=2,padding='same',
                        input_shape=(X_train.shape[1],X_train.shape[2])),
        K.layers.MaxPooling1D(3),
        K.layers.Dropout(DROPRATE),
        K.layers.BatchNormalization(),
        K.layers.Conv1D(256, 3, activation='relu',strides=2,padding='same',
                        input_shape=(X_train.shape[1],X_train.shape[2])),
        K.layers.MaxPooling1D(3),
        K.layers.Dropout(DROPRATE),
        K.layers.BatchNormalization(),
        K.layers.Conv1D(256, 3, activation='relu',strides=2,padding='same',
                        input_shape=(X_train.shape[1],X_train.shape[2])),
        K.layers.GlobalMaxPooling1D(),
        K.layers.Dropout(DROPRATE),
        K.layers.BatchNormalization(),
        K.layers.Dense(256,activation='relu'),
        K.layers.Dense(1,activation='linear')])
    opt=K.optimizers.Adam(lr=learning_rate, decay=learning_rate/epochs)
    model.compile(optimizer=opt, loss=penalized_loss,metrics=[penalized_loss])
    sampler=UnderSampler3D(X_train,y_train,batch_size=batch_size)
    model.fit_generator(sampler,
                        shuffle=False,
                        epochs=epochs,
                        steps_per_epoch=steps,
                        validation_data=(X_dev,y_dev),
                        callbacks=[tensorboard_callback,cp_callback])
    return model

In [ ]:
model=experiment(learning_rate=0.02,epochs=1000,batch_size=128,steps=200,name='model')

W0826 10:22:33.615246 140247743416128 callbacks.py:862] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Epoch 1/1000
200/200 [==============================] - 10s 50ms/step - loss: 1.7685 - penalized_loss: 1.7685 - val_loss: 0.8540 - val_penalized_loss: 0.8540
Epoch 2/1000
200/200 [==============================] - 8s 41ms/step - loss: 0.4561 - penalized_loss: 0.4561 - val_loss: 0.5886 - val_penalized_loss: 0.5886
Epoch 3/1000
200/200 [==============================] - 9s 47ms/step - loss: 0.4131 - penalized_loss: 0.4131 - val_loss: 0.4247 - val_penalized_loss: 0.4247
Epoch 4/1000
200/200 [==============================] - 8s 42ms/step - loss: 0.4095 - penalized_loss: 0.4095 - val_loss: 0.7406 - val_penalized_loss: 0.7406
Epoch 5/1000
200/200 [==============================] - 9s 45ms/step - loss: 0.3849 - penalized_loss: 0.3849 - val_loss: 0.3670 - val_penalized_loss: 0.3670
Epoch 6/1000
200/200 [==============================] - 14s 71ms/step - loss: 0.3733 - penalized_loss: 0.3733 - val_loss: 0.3831 - val_penalized_loss: 0.3831
Epoch 7/1000
200/200 [==============================] - 

W0826 10:30:54.747246 140247743416128 deprecation.py:506] From /home/kvassay/.virtualenvs/main/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


200/200 [==============================] - 12s 61ms/step - loss: 0.2568 - penalized_loss: 0.2568 - val_loss: 0.3561 - val_penalized_loss: 0.3561
Epoch 51/1000
200/200 [==============================] - 14s 69ms/step - loss: 0.2530 - penalized_loss: 0.2530 - val_loss: 0.2514 - val_penalized_loss: 0.2514
Epoch 52/1000
200/200 [==============================] - 10s 49ms/step - loss: 0.2414 - penalized_loss: 0.2414 - val_loss: 0.2860 - val_penalized_loss: 0.2860
Epoch 53/1000
200/200 [==============================] - 9s 44ms/step - loss: 0.2469 - penalized_loss: 0.2469 - val_loss: 0.2661 - val_penalized_loss: 0.2661
Epoch 54/1000
200/200 [==============================] - 10s 48ms/step - loss: 0.2428 - penalized_loss: 0.2428 - val_loss: 0.2962 - val_penalized_loss: 0.2962
Epoch 55/1000
200/200 [==============================] - 9s 44ms/step - loss: 0.2421 - penalized_loss: 0.2421 - val_loss: 0.3050 - val_penalized_loss: 0.3050
Epoch 56/1000
200/200 [==============================] - 9s 45

In [ ]:
model.summary()

In [ ]:
## Persist

In [ ]:
model.save(MODEL_SAVE_DIR)